In [43]:
import os
import pandas as pd
from pathlib import Path
from pandas import DataFrame

In [44]:
%reload_ext dotenv
%dotenv

In [45]:
BASE_PATH               = os.getenv('BASE_PATH')
REGEX_DATA              = os.getenv('REGEX_DATA')
INDICE_NAME             = os.getenv('INDICE_NAME')
ENGINE_NAME             = os.getenv('ENGINE_NAME')
NOME_INSTITUICAO        = os.getenv('NOME_INSTITUICAO')
NOME_COLUNA_DATA        = os.getenv('NOME_COLUNA_DATA')
NOME_COLUNA_SALDO       = os.getenv('NOME_COLUNA_SALDO')
DECIMAL_INDICATOR       = os.getenv('DECIMAL_INDICATOR')
FILE_TYPE_TO_IMPORT     = os.getenv('FILE_TYPE_TO_IMPORT')
FILE_TYPE_TO_EXPORT     = os.getenv('FILE_TYPE_TO_EXPORT')
NOME_COLUNA_LANCAMENTO  = os.getenv('NOME_COLUNA_LANCAMENTO')
NOME_COLUNA_INSTITUICAO = os.getenv('NOME_COLUNA_INSTITUICAO')
IMPORTED_FILE_SEPARATOR = os.getenv('IMPORTED_FILE_SEPARATOR')
VALUES_TO_IGNORE        = os.getenv('VALUES_TO_IGNORE').split(",")

In [46]:
def convert_dataframe_to_notion_format(
        df: DataFrame,
        nome_instituicao: str,
        coluna_data: str,
        coluna_saldo: str,
        coluna_instituicao: str,
        file_uses_day_first = True
    ) -> DataFrame:
    df[coluna_instituicao] = nome_instituicao
    df[coluna_saldo] = pd.to_numeric(df[coluna_saldo])
    df[coluna_data] = pd.to_datetime(df[coluna_data], dayfirst=file_uses_day_first).dt.strftime(REGEX_DATA)
    return df

In [47]:
def filter_dataframe(
        df: DataFrame,
        coluna: str,
        values_to_ignore
    ) -> DataFrame:
    return df[~df[coluna].isin(values_to_ignore)]

In [48]:
def get_file_name_to_save(
        df: DataFrame,
        coluna: str,
        indice: int,
        type: str
    ) -> str:
    value = df[coluna][indice]
    return f'{value[-4:]}-{value[0:2]}.{type}'

In [49]:
def save_dataframe_to_csv(
        df: DataFrame,
        coluna: str,
        indice: str,
        file_type: str,
        show_index = False
    ):
    filename = get_file_name_to_save(df, coluna, int(indice), file_type)
    df.to_csv(f'{filename}', index=show_index)

In [50]:
for file in os.listdir(BASE_PATH):
    if file.endswith(FILE_TYPE_TO_IMPORT):
        df = pd.read_csv(
            Path(file),
            sep=IMPORTED_FILE_SEPARATOR,
            engine=ENGINE_NAME,
            names=[NOME_COLUNA_DATA, NOME_COLUNA_LANCAMENTO, NOME_COLUNA_SALDO],
            decimal=DECIMAL_INDICATOR
        )

        if df.empty:
            continue
        else:
            df = convert_dataframe_to_notion_format(
                df,
                NOME_INSTITUICAO,
                NOME_COLUNA_DATA,
                NOME_COLUNA_SALDO,
                NOME_COLUNA_INSTITUICAO
            )
            filtered_df = filter_dataframe(
                df,
                NOME_COLUNA_LANCAMENTO,
                VALUES_TO_IGNORE
            )
            save_dataframe_to_csv(
                filtered_df,
                NOME_COLUNA_DATA,
                INDICE_NAME,
                FILE_TYPE_TO_EXPORT
            )